# RAG with Milvus

This notebook examines how to do basic RAG in [Milvus](https://milvus.io/) vector DB.

**NOTE: Please check `rag.ipynb` first!**

## Prerequisites

1. `ollama` server
2. `ollama`: Ollama Python Library.
3. `langchain`
4. `pymilvus`: Milvus is a vector DB.
5. `pymilvus[model]`: for default Milvus embedding model.

In [1]:
# llm_name = 'deepseek-r1:7b'
llm_name = 'gemma3:12b'
ollama_hostname = 'http://localhost:11434'
file_path = 'files/elden_ring.md'

## Preprocessing External Files

We will work on the same external markdown file for this demo.

### Load File & Split Text

In [2]:
def load_file_content(file_path):
    with open(file_path, encoding='utf-8') as f:
        return f.read()

In [3]:
file_content = load_file_content(file_path)
print(file_content[:min(150, len(file_content))])

# Elden Ring: A Comprehensive Guide

**Elden Ring** is an action RPG developed by FromSoftware and published by Bandai Namco Entertainment. Directed b


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [5]:
chunks = text_splitter.split_text(file_content)

In [6]:
# debug sliced chunks
print([len(chunk) for chunk in chunks])
if len(chunks) >= 2:
    display(chunks[0][-100:])
    print('<SEPARATOR>')
    display(chunks[1][:100])

[365, 442, 488, 485, 388, 399, 435, 326, 316, 363, 285, 257, 296, 332, 414, 420, 477, 329, 261, 430, 392]


'gameplay FromSoftware is renowned for with a vast, open-world experience.\n\n---\n\n## Story and Setting'

<SEPARATOR>


'---\n\n## Story and Setting\n\n**Elden Ring** is set in the mystical world of the Lands Between, where t'

### Embed Text & Store to Vector DB

Embed the chunks using an embedding model.

In [7]:
from pymilvus import model

# default: DefaultEmbeddingFunction = dense.onnx.OnnxEmbeddingFunction
embedding_fn = model.DefaultEmbeddingFunction()

def embed_txts(txts: list[str]) -> list:
    return embedding_fn.encode_documents(txts)

/root/miniconda3/envs/edge-ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
embeddings = embed_txts(chunks)

In [9]:
print(type(embeddings))
print(type(embeddings[0]))
print(embeddings[0].shape)
embedding_dim_size = embeddings[0].shape[0]
embedding_dim_size

<class 'list'>
<class 'numpy.ndarray'>
(768,)


768

Now create a vector DB and store the embeddings as well as the original chunks.

In [10]:
from pymilvus import MilvusClient

milvus_client = MilvusClient('milvus_demo.db')

In [11]:
demo_collection_name = 'demo_collection'

if milvus_client.has_collection(demo_collection_name):
    milvus_client.drop_collection(demo_collection_name)
milvus_client.create_collection(
    collection_name=demo_collection_name,
    dimension=embedding_dim_size,
)

In [12]:
insert_data = [
    {
        'id': i,
        'vector': embeddings[i],
        'text': chunk,
    }
    for i, chunk in enumerate(chunks)
]

In [13]:
milvus_client.insert(demo_collection_name, data=insert_data)

{'insert_count': 21, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'cost': 0}

## Specify RAG Logic

In [14]:
import ollama

ollama_client = ollama.Client(host=ollama_hostname)

In [15]:
prompt_template = """
I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: {question}
"""

In [16]:
# answer questions using RAG
def answer(questions: list[str], stream=False) -> str:
    # embed query
    embedded_queries = embed_txts(questions)
    # retrieve the most relevant document chunks
    # https://milvus.io/api-reference/pymilvus/v2.5.x/MilvusClient/Vector/search.md
    search_res = milvus_client.search(
        demo_collection_name,
        anns_field='vector',
        data=embedded_queries,
        limit=3,
        search_params={'metric_type': "COSINE"},
        output_fields=['text']
    )
    # format context
    combined_docs = []
    for cur_q in search_res:
        cur_combined_doc = ''
        for i, cur_res in enumerate(cur_q):
            cur_combined_doc += f'Chunk {i}:\n{cur_res['entity']['text']}'
            if i != len(cur_q) - 1:
                cur_combined_doc += '\n'
        combined_docs.append(cur_combined_doc)
    # format prompt
    enhanced_q = [
        f'Relevant text chunks retrieved from the vector DB:\n{cur_doc}\nAnswer this question: {questions[q_i]}'
        for q_i, cur_doc in enumerate(combined_docs)
    ]
    formatted_prompts = [prompt_template.format(question=cur_x_q) for cur_x_q in enhanced_q]
    # execute
    answers = []
    for cur_prompt in formatted_prompts:
        print('===== QUESTION =====')
        print(cur_prompt)
        print('===== GENERATE =====')
        resp = ollama_client.generate(model=llm_name, prompt=cur_prompt, stream=stream)
        cur_answer = ''
        if stream:
            for chunk in resp:
                cur_answer += chunk.response
                print(chunk.response, end='', flush=True)
        else:
            cur_answer = resp.response
            print(cur_answer)
        answers.append(cur_answer)
    return answers

## Test

### Test 0

In [17]:
res = answer(['What is Elden Ring?'], stream=True)

===== QUESTION =====

I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: Relevant text chunks retrieved from the vector DB:
Chunk 0:
---

# Conclusion

Elden Ring stands as a monumental achievement in the action RPG genre. Its vast open world, intricate lore, and challenging gameplay offer an unparalleled experience for fans and newcomers alike. Whether you're delving into the lore, mastering combat mechanics, or exploring every nook and cranny of the Lands Between, Elden Ring provides a rich, rewarding adventure that sets a new standard for the genre.

---
Chunk 1:
# Elden Ring: A Comprehensive Guide

**Elden Ring** is an action RPG developed by FromSoftware and published by Bandai Namco Entertainment. Directed by Hidetaka Miyazaki, in collaboration with George R. R. Martin, the game blends the intricate lore and challenging gameplay FromSoftware is re

## Test 1

In [18]:
res = answer(['Does the game support multiplayer mode?'], stream=True)

===== QUESTION =====

I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: Relevant text chunks retrieved from the vector DB:
Chunk 0:
---

## Multiplayer and Online Features

**Co-op and PvP:**
- **Co-op**: Players can summon others to help with tough areas or bosses, making the game more manageable.
- **PvP**: Competitive multiplayer allows players to invade others' worlds, adding an element of risk and excitement.
Chunk 1:
---

## Gameplay Mechanics

**Combat and Exploration:**
- **Combat**: The combat system in Elden Ring builds on FromSoftware's previous titles, emphasizing timing, strategy, and resource management. Players can choose from a variety of weapons, magic, and abilities to tailor their combat style.
- **Exploration**: The open-world design encourages exploration. Players can traverse the Lands Between on foot or horseback, discovering hid

### Test 2 

In [19]:
res = answer(['Who produced the soundtrack for the game?'], stream=True)

===== QUESTION =====

I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: Relevant text chunks retrieved from the vector DB:
Chunk 0:
---

## Technical and Artistic Aspects

**Graphics and Art Style:**
- **Visuals**: Stunning graphics that blend gothic and high fantasy elements.
- **Art Direction**: A unique art style that enhances the game's atmosphere and storytelling.

**Audio:**
- **Soundtrack**: Composed by Yuka Kitamura and others, the music enhances the mood and intensity of the game.
- **Voice Acting**: High-quality voice acting brings characters and lore to life.

---

## Development and Release
Chunk 1:
---

## Development and Release

**History:**
- **Announcement**: First revealed in 2019, developed by FromSoftware and directed by Hidetaka Miyazaki, with world-building by George R. R. Martin.
- **Release Date**: Released on February 25, 2022,